In [299]:
import geopandas as gpd
import pandas as pd

In [468]:
censusRaceWithGeometry = gpd.read_file("censusRaceWithGeometry.geojson")
censusRaceWithGeometry["roundedWhitePercep"] = censusRaceWithGeometry.whitePercep.rank(pct=True)//(1/10)

In [469]:
censusRaceWithGeometry[["roundedWhitePercep"]].value_counts()

roundedWhitePercep
0.0                   142
2.0                   133
3.0                   133
4.0                   133
5.0                   133
6.0                   133
7.0                   133
8.0                   133
9.0                   133
1.0                   123
dtype: int64

In [470]:
pointsWithSeg = gpd.read_file("pointsWithSeg.geojson")
pointsWithSeg = pointsWithSeg.filter(regex="^((?!_q).)*$")

In [471]:
pointsWithSeg.head()

,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice,geometry
0,1,420708,000001,0.030779,0.003141,0.163661,0.020548,0.118350,0.033204,0.291897,0.201790,0.017978,0.000147,POINT (-75.16460 39.95994)
1,2,422065,000002,0.047397,0.008571,0.367944,0.009722,0.108147,0.035982,0.209341,0.089528,0.003407,0.000034,POINT (-75.16357 39.96050)
2,3,420702,000003,0.034286,0.003337,0.178823,0.008427,0.237291,0.061395,0.024113,0.378118,0.006196,0.000012,POINT (-75.16619 39.96013)
3,5,420696,000004,0.019361,0.001505,0.331703,0.004485,0.274849,0.073919,0.099125,0.107871,0.027312,0.000077,POINT (-75.16776 39.96033)
4,8,422066,000005,0.051927,0.002919,0.139440,0.039439,0.035637,0.003948,0.092139,0.446469,0.020049,0.000054,POINT (-75.16672 39.96089)


In [472]:
censusRace2 = censusRaceWithGeometry[['tract','block group','geometry','roundedWhitePercep']]
censusRace2.head()

,tract,block group,geometry,roundedWhitePercep
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0
1,009500,3,"POLYGON ((-75.24043 39.96354, -75.24011 39.965...",0.0
2,028200,1,"POLYGON ((-75.14683 40.02960, -75.14671 40.030...",1.0
3,033702,2,"POLYGON ((-75.06981 40.07086, -75.06923 40.071...",7.0
4,037200,2,"POLYGON ((-75.16508 39.91337, -75.16486 39.914...",8.0


In [473]:
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]

In [474]:
censusRace_join = gpd.sjoin(censusRace2,pointsWithSeg,op='intersects', how='left')
censusRace_join.head()

,tract,block group,geometry,roundedWhitePercep,index_right,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0,9079.0,19157.0,420667.0,009080,0.026769,0.000596,0.241496,0.049876,0.133667,0.004428,0.044889,0.299683,0.019361,0.000187
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0,9054.0,19115.0,420868.0,009055,0.007815,0.001313,0.125098,0.031721,0.233294,0.010317,0.154183,0.293472,0.047849,0.000079
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0,9052.0,19113.0,420867.0,009053,0.038071,0.000508,0.119719,0.002063,0.264504,0.070930,0.203647,0.198384,0.012682,0.000212
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0,9078.0,19152.0,422140.0,009079,0.015346,0.005183,0.213215,0.007887,0.264627,0.007511,0.050901,0.327638,0.079062,0.000052
0,009100,3,"POLYGON ((-75.20223 39.96164, -75.20142 39.961...",3.0,9082.0,19162.0,421033.0,009083,0.015580,0.002593,0.268059,0.023788,0.091601,0.036865,0.116603,0.253316,0.044529,0.000026


In [475]:
Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
readyToPlot = censusRace_join.groupby('roundedWhitePercep').mean(Vars)
readyToPlot = readyToPlot.drop(columns=['index_right','fid','SEG_ID'])
readyToPlot = readyToPlot.reset_index()

readyToPlot[Vars] = readyToPlot[Vars].apply(lambda x: (x-x.min())/(x.max()-x.min()))

readyToPlot = pd.melt(readyToPlot, id_vars=['roundedWhitePercep'])
readyToPlot['roundedWhitePercep'] = readyToPlot.roundedWhitePercep.astype(str)

In [476]:
readyToPlot["valueSquare"]= readyToPlot.value**2
readyToPlot.head(20)

,roundedWhitePercep,variable,value,valueSquare
0,0.0,wall,1.000000,1.000000
1,1.0,wall,0.999696,0.999392
2,2.0,wall,0.746241,0.556875
3,3.0,wall,0.841182,0.707586
4,4.0,wall,0.692188,0.479124
5,5.0,wall,0.551354,0.303991
6,6.0,wall,0.114220,0.013046
7,7.0,wall,0.102787,0.010565
8,8.0,wall,0.000000,0.000000
9,9.0,wall,0.532632,0.283697


In [477]:
import altair as alt
from vega_datasets import data

selection = alt.selection_multi(fields=['variable'], bind='legend')

chart = alt.Chart(readyToPlot).mark_area(point = True,tooltip = True,interpolate="natural"
).encode(
    x=alt.X('roundedWhitePercep:N',axis=alt.Axis(ticks=False, domain=False,labelAngle=0)),
    y=alt.Y('value:Q',axis=alt.Axis(ticks=False, domain=False)),
    row='variable:N',
    color=alt.condition(selection,
                        alt.Color('variable:N', scale=alt.Scale(scheme='category20b')),
                        alt.value('lightgray')),   
    opacity=alt.condition(selection, alt.value(0.6), alt.value(0.3))
).configure_view(strokeWidth=0
).properties(height=100,width=800).add_selection(selection).interactive()

chart.encoding.x.title = 'White Percentage'
chart.encoding.y.title = 'Value'

chart

alt.Chart(...)